In [1]:
from PIL import Image
import numpy as np
from tensorflow.python.keras.models import load_model

In [2]:
classes = ['skunk','fox','rodent','dog','squirrel','cat','rabbit','bird','cow','bobcat','deer','raccoon','coyote','opossum','other']
classes_dict_lookup = dict(zip(range(15), classes))

In [6]:
model = load_model('edge/model/combined_model.h5')

In [5]:
def load_image(path):
    img = Image.open(path)
    img = img.resize(size=(299,299), resample=Image.LANCZOS).convert("RGB")
    img = np.array(img)
    img = img / 255.0
    # Convert 2-dim gray-scale array to 3-dim RGB array.
    if (len(img.shape) == 2):
        img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
    return img

In [6]:
predicted_id = model.predict(load_image('data/cct_images/59439fd8-23d2-11e8-a6a3-ec086b02610b.jpg').reshape(1, 299, 299, 3))
predicted_name = classes_dict_lookup[predicted_id.argmax()]
predicted_name


'coyote'

In [6]:
img_array = load_image('data/cct_images/59439fd8-23d2-11e8-a6a3-ec086b02610b.jpg').reshape(1, 299, 299, 3)

In [8]:
img_array.shape

(1, 299, 299, 3)

In [ ]:
im = Image.fromarray(img_array)
im.save("your_file.jpeg")

# Tensorflow Lite and Quantization

In [4]:
import tensorflow as tf

In [5]:
tf.__version__

'2.0.0'

In [21]:
tf.saved_model.save(model,'edge/model/')

INFO:tensorflow:Assets written to: edge/model/assets


In [14]:
converter = tf.lite.TFLiteConverter.from_saved_model('edge/model/')

INFO:tensorflow:Restoring parameters from edge/model/variables/variables
INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'__saved_model_init_op', 'serving_default'}
INFO:tensorflow:input tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: input_17
INFO:tensorflow: tensor name: serving_default_input_17:0, shape: (-1, 299, 299, 3), type: DT_FLOAT
INFO:tensorflow:output tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: dense_30
INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 15), type: DT_FLOAT
INFO:tensorflow:Restoring parameters from edge/model/variables/variables
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 380 variables.
INFO:tensorflow:Converted 380 variables to const ops.


In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [9]:
tflite_model = converter.convert()

In [10]:
open("edge/model/converted_model.tflite", "wb").write(tflite_model)

89227712

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quantized_model)